In [1]:
from tensorflow.keras.models import load_model
from tensorflow.nn import softmax
import cv2,numpy as np,time

In [2]:
# Check for Available Cameras
available_cameras = [*filter(lambda x:not(cv2.VideoCapture(x) is None or not cv2.VideoCapture(x).isOpened()),range(6))]
available_cameras

[0, 2]

In [ ]:
cap = cv2.VideoCapture(available_cameras[0])

mainFrame = np.zeros(550*600*3,dtype=np.uint8).reshape(550,600,3)
model = load_model("sixtyfour.h5")
types = ["up","down","left","right","front","back","land","flip","land"]


while True:
    _ , frame = cap.read()
    frame = frame[:,79:559,:]

        pts , raw = convertToCords(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB),1)
    pickim = showImg(frame,raw)
    conf=0
    if len(raw.handedness)!=0:
        preds = model.predict(np.array(pts).reshape(1,43),verbose=0)
        softout = softmax(preds)
        ans = types[np.argmax(softout)]
        conf = np.max(softout.numpy())
        print(ans ,conf, list(softout.numpy().tolist()) ,list(pts),end="\r")
    
    mainFrame[70:,79:559,:] = pickim
    if conf > 0.5:
        mainFrame[:70,:300,:] = np.zeros(70*300*3).reshape(70,300,3)
        cv2.putText(mainFrame,f"{ans} {conf}",[20,20],0,0.5,color=[0,255,0])
    else:
        mainFrame[:70,:300,:] = np.zeros(70*300*3).reshape(70,300,3)
        cv2.putText(mainFrame,f"no strong detection",[20,20],0,0.5,color=[0,0,255])

    cv2.imshow("Frame",mainFrame)

    if (cv2.waitKey(10) & 0xFF == ord('q')):
        break
cap.release()
cv2.destroyAllWindows()